### SQL Agents

In [17]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [3]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
llm = ChatOpenAI(
    model=model_name,
    openai_api_key=openai_api_key,
    temperature=0.1,
    max_tokens=1000
)

In [4]:
db_file_path = "./chinook.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

In [5]:
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [6]:
query = "SELECT * FROM Artist LIMIT 10;"

db.run(query)

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [7]:
chain = create_sql_query_chain(llm, db)

In [8]:
response = chain.invoke({
    "question": "How many employees are there in the organization?"
})

print(response)

SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"


In [9]:
db.run(response)

'[(8,)]'

In [11]:
@tool
def parse(query_string: str) -> str:
    """
    Parses Generated SQL Statement
    """

    splitted = query_string.split(":")

    if (len(splitted) >= 2):
        query = splitted[1]
    else:
        query = query_string

    return query

In [12]:
# GPT 4 Generated SQL Statement

query = """
QUERY: SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"
"""

In [13]:
print(parse.invoke(query))


 SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"



In [14]:
# GPT 3.5 Generated SQL Statement
query = """
SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"
"""

print(parse.invoke(query))


SELECT COUNT("EmployeeId") AS "TotalEmployees"
FROM "Employee"



In [15]:
execute_query = QuerySQLDataBaseTool(db = db)
write_query = create_sql_query_chain(llm, db)

chain = write_query | parse | execute_query

In [16]:
question = "How many employees are there in the organization?"

response = chain.invoke({
    "question": question
})

print(response)

[(8,)]


In [18]:
answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding SQL Query, and SQL Result, answer the user question.

    Question: {question}
    SQL Query: {query}
    Result: {result}
    Answer:
    """
)

In [19]:
chain = RunnablePassthrough.assign(query = write_query) \
    .assign(result = itemgetter("query") | parse | execute_query) | \
    answer_prompt | llm | StrOutputParser()

In [20]:
question = "How many employees are there in the organization?"

response = chain.invoke({
    "question": question
})

print(response)

There are 8 employees in the organization.


In [21]:
question = "How many employees are there in each of the organization?"

response = chain.invoke({
    "question": question
})

print(response)

General Manager: 1 employee
IT Manager: 1 employee
IT Staff: 2 employees
Sales Manager: 1 employee
Sales Support Agent: 3 employees


In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langgraph.prebuilt import create_react_agent #reasoning and action
from langchain_core.messages import HumanMessage, SystemMessage

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F8EC8F60C0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F8EC8F60C0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F8EC8F60C0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [25]:
SQL_PREFIX = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""

In [26]:
system_message = SystemMessage(content=SQL_PREFIX)
agent_executor = create_react_agent(
    llm, tools, messages_modifier=system_message
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_7268\2816356074.py:2: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(


In [27]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="which country's customers have spent the most?")
    ]
}):
    print(stream)
    print("******")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KwM16CaDf9X2W83um1DbzG3v', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 559, 'total_tokens': 571, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5adcc192-4506-4f25-a387-aac8089f0f20-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_KwM16CaDf9X2W83um1DbzG3v', 'type': 'tool_call'}], usage_metadata={'input_tokens': 559, 'output_tokens': 12, 'total_tokens': 571, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [28]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Describe the playlisttrack table.")
    ]
}):
    print(stream)
    print("******")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_u94Cet4Qw4P2dgIz1ncd3YOX', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 556, 'total_tokens': 573, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-01661eee-6bbf-4893-ba41-0a06f49f87dc-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_u94Cet4Qw4P2dgIz1ncd3YOX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 556, 'output_tokens': 17, 'total_tokens': 573, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'outpu

In [29]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Top 5 customers who have spent most buying MPEG Audio, and heavy metal type of playlists.")
    ]
}):
    print(stream)
    print("******")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lM9vzuECyvMCELTyqNfyTXSe', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 569, 'total_tokens': 581, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6baf9bf0-547f-4f83-9354-2842420c5f61-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_lM9vzuECyvMCELTyqNfyTXSe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 569, 'output_tokens': 12, 'total_tokens': 581, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]